## Reexecução

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Dict das tags

In [1]:
%%time
#reference dic
f = open('/home/lzirondi/Github/snorkel/Scripts/LIWC2015_pt_desc_only.dic', 'r', encoding='utf-8-sig')
s = f.readlines()
f.close()

tags = dict()

for line in s:
    line = line.split('\t')
    tags[line[0]] = line[1].replace('\n', '')
    
print(tags)

{'1': 'function (Function Words)', '2': 'pronoun (Pronouns)', '3': 'ppron (Personal Pronouns)', '4': 'i (I)', '5': 'we (We)', '6': 'you (You)', '7': 'shehe (SheHe)', '8': 'they (They)', '9': 'ipron (Impersonal Pronouns)', '10': 'article (Articles)', '11': 'prep (Prepositions)', '12': 'auxverb (Auxiliary Verbs)', '13': 'adverb (Adverbs)', '14': 'conj (Conjunctions)', '15': 'negate (Negations)', '20': 'verb (Verbs)', '21': 'adj (Adjectives)', '22': 'compare (Comparisons)', '23': 'interrog (Interrogatives)', '24': 'number (Numbers)', '25': 'quant (Quantifiers)', '30': 'affect (Affect)', '31': 'posemo (Positive Emotions)', '32': 'negemo (Negative Emotions)', '33': 'anx (Anx)', '34': 'anger (Anger)', '35': 'sad (Sad)', '40': 'social (Social)', '41': 'family (Family)', '42': 'friend (Friends)', '43': 'female (Female)', '44': 'male (Male)', '50': 'cogproc (Cognitive Processes)', '51': 'insight (Insight)', '52': 'cause (Causal)', '53': 'discrep (Discrepancies)', '54': 'tentat (Tentative)', '55

## Dict de palavras do LIWC

In [ ]:
%%time
f = open('/home/lzirondi/Github/snorkel/Scripts/LIWC2015_pt_no_desc.dic', 'r', encoding='utf-8-sig')

s = f.readlines()
f.close()

liwc = dict()

#list[1:]

for line in s:
    tags2 = []
    line = line.split('\t')
    for element in line:
        if element != '' and element != '\n':
            tags2.append(element)
    
    liwc[tags2[0]] = tags2[1:]
print(liwc)

In [15]:
d = liwc.copy()
print(d)
#for key in d:
#    print(key + ' ' + d[key])

{'(:': ['affect (Affect)', 'posemo (Positive Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], '):': ['affect (Affect)', 'negemo (Negative Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':(': ['affect (Affect)', 'negemo (Negative Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ":'(": ['affect (Affect)', 'negemo (Negative Emotions)', 'sad (Sad)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':-(': ['affect (Affect)', 'negemo (Negative Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':((': ['affect (Affect)', 'negemo (Negative Emotions)', 'sad (Sad)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':)': ['affect (Affect)', 'posemo (Positive Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':-)': ['affect (Affect)', 'posemo (Positive Emotions)', 'informal (Informal Language)', 'netspeak (Netspeak)'], ':/': ['affect (Affect)', 'negemo (Negative Emotions)', 'informal (Inform

In [ ]:
%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = [session.query(relation).filter(relation.split == 0).all(), session.query(relation).filter(relation.split == 1).all(), session.query(relation).filter(relation.split == 2).all()]

sw = set(stopwords.words("portuguese"))

quant = dict()

for group in cands:
    for candidate in group:
        for word in get_between_tokens(candidate):
            if word not in sw:
                if word in quant:
                    quant[word] = quant[word] + 1
                else:
                    quant[word] = 1
    

words = []

for key in quant:
    aux = [key,quant[key]]
    words.append(aux)
    
from operator import itemgetter

words = sorted(words, key=itemgetter(1,0))


## Criando os arquivos dos dicionários

In [ ]:
import Preprocessing as prep

inPath = ''
outPath = ''
corpusPath = ''
LIWCpath = '/home/lzirondi/Github/snorkel/Scripts/'

util = prep.Util(inPath, outPath, LIWCpath, False)

util.LIWKdicios()

## Criando os dicionários

In [ ]:
paths = ['/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Verbos',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas & Verbos'
]

causas = {}
verbos = {}
causas_e_verbos = {}

dicios = {
    paths[0] : causas,
    paths[1] : verbos,
    paths[2] : causas_e_verbos
}

for file in paths:
    f = open(file, 'r', encoding='UTF-8')
    s = f.readlines()
    f.close()
    
    for word in s:
        word = word.replace('\n', '')
        dicios.get(file)[word] = 0    

## Análise

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dicios = [causas, verbos, causas_e_verbos]

#chegar no candidato e ver se entre os tokens tem algo que a gente quer

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw:
            print(word)
            aux = False
            if causas.get(word):
                causas[word] = causas[word] + 1
                print(word)
                if verbos.get(word):
                    aux = True
                    verbos[word] = verbos[word] + 1
                    causas_e_verbos[word] = causas_e_verbos[word] + 1

            if(aux):
                print(word)
                verbos[word] = verbos[word] + 1

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords


import re

numero = re.compile('\D+')
repeticao = re.compile('\w{2,}')

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dic = {}

#nao repetir frases
#ignorar palavras de 1 letra 

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw and len(word)>1 and numero.match(word):
            if dic.get(word):
                dic[word] = dic[word] + 1
            else:
                dic[word] = 1
                
ordenada = []
maior = ['', 0]

dici = dic


for i in range(0, len(dici)):
    for key in dici:
        if dici[key] > maior[1]:
            maior[0] = key
            maior[1] = dici[key]
    ordenada.append(maior)
    dici.pop(maior[0])
    maior = ['', 0]
                

f = open('/home/lzirondi/Github/snorkel/Scripts/Análise de Palavras/between_tokens_stopFiltered.tsv', 'w', encoding='UTF-8')

for word in ordenada:
    f.write(word[0] + '\t' + str(word[1]) + '\n')
    
f.close()

## Testes

In [ ]:
cands = session.query(relation).filter(relation.split == 0).all()

from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()
#print(session.query(relation).filter(relation.split == 1))


#candi = cands[0]

print(type(candi))
print(cands[0].id)
for candi in cands:
    print(candi.id)

In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print(len(cands))

rep = []

for candidate in cands:
    s = candidate.get_parent().stable_id.split(':')[0]
    
    if s not in rep:
        rep.append(candidate.id)

print(len(rep))
print(len(cands))
#print(dir(cands[0]))
#print(cands[0].get_parent())

print(cands[0].get_parent().stable_id)


In [ ]:
d = dict()

d['a'] = 'a'


try:
    print(d['a'])
except:
    pass